<a href="https://colab.research.google.com/github/tripathishubham1/Automated-Essay-Grading-Machine/blob/main/EGM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Automated Essay Grading Machine**

## Importing Modules & Dataset Manipulation

In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')



In [2]:
df_train

,Unnamed: 0,promptId,uniqueId,essay,evaluator_rating
0,0,1,1_323,"At present age, our education system is not go...",3.0
1,1,1,1_238,I am agree the tightly defined curriculum of o...,4.0
2,2,1,1_212,I strongly agree with the statement that tight...,2.0
3,3,1,1_117,Our education system is nice quitely but i dis...,2.0
4,4,1,1_229,i am totally agree with the statement that tig...,3.0
...,...,...,...,...,...
1235,1235,5,5_419,The entire world is in the race of producing a...,3.0
1236,1236,5,5_420,The race in the development of weapons are pro...,2.5
1237,1237,5,5_421,In an era where every second person hopes and ...,4.0
1238,1238,5,5_422,INTRODUCTION :Since the beginning of the time ...,3.0


##Preprocesing Of Text Data

In [3]:
train_score = df_train['evaluator_rating']
train_e = df_train['essay'].tolist()
test_e = df_test['essay'].tolist()
nltk.download("stopwords")
nltk.download('punkt')
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english'))

def fun1(x):
  x = re.sub("[^A-Za-z]"," ",x)
  x.lower()
  fil_sen = []
  words = x.split()
  for w in words:
    if w not in stop_words:
      fil_sen.append(w)
  return fil_sen

def fun2(essay):
  essay = essay.strip()
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  raw = tokenizer.tokenize(essay)
  final_words = []
  for i in raw:
    if len(i)>0:
      final_words.append(fun1(i))
  return final_words

for i in train_e:
  train_sents+=fun2(i)
for i in test_e:
  test_sents+=fun2(i)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##Vectorization & Applying LSTM Model

In [4]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index2word)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model[i])        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(fun1(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(fun1(i))
testing_vectors = getVecs(clean_test, model, num_features)

training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 300)            721200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


##Fitting Model & Predicting the Output of Test Dataset

In [5]:
lstm_model.fit(training_vectors, train_score, batch_size=155, epochs=100,validation_split=0.5)

Epoch 1/100
4/4 [==============================] - 34s 867ms/step - loss: 7.6674 - mae: 2.5810 - val_loss: 6.1969 - val_mae: 2.3114
Epoch 2/100
4/4 [==============================] - 0s 60ms/step - loss: 4.8519 - mae: 2.0346 - val_loss: 2.5241 - val_mae: 1.4765
Epoch 3/100
4/4 [==============================] - 0s 59ms/step - loss: 1.7956 - mae: 1.1370 - val_loss: 1.2815 - val_mae: 0.9243
Epoch 4/100
4/4 [==============================] - 0s 66ms/step - loss: 1.2400 - mae: 0.8587 - val_loss: 1.2251 - val_mae: 0.8685
Epoch 5/100
4/4 [==============================] - 0s 58ms/step - loss: 1.0439 - mae: 0.7953 - val_loss: 1.2275 - val_mae: 0.8720
Epoch 6/100
4/4 [==============================] - 0s 59ms/step - loss: 1.1834 - mae: 0.8406 - val_loss: 1.2222 - val_mae: 0.8650
Epoch 7/100
4/4 [==============================] - 0s 56ms/step - loss: 1.2202 - mae: 0.8455 - val_loss: 1.2067 - val_mae: 0.8369
Epoch 8/100
4/4 [==============================] - 0s 56ms/step - loss: 1.1183 - mae: 0.

In [6]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

array([[3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [2.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [2.],
       [3.],
       [2.],
       [3.],
       [3.],
       [3.],
       [2.],
       [2.],
       [3.],
       [3.],

In [7]:
df_test['predicted_score'] = y_pred.tolist()

In [8]:
df_test

,Unnamed: 0,promptId,uniqueId,essay,predicted_score
0,0,1,1_315,Curriculum has been adopted in many schools. T...,[3.0]
1,1,1,1_214,"I strongly agree with the statement , The tig...",[3.0]
2,2,1,1_196,Imagination and creativity is the most importa...,[3.0]
3,3,1,1_178,In our eduction system leaves no room for imag...,[3.0]
4,4,1,1_201,"I will agree at some what extend, because if w...",[3.0]
...,...,...,...,...,...
300,300,5,5_146,Earth is a creation of God and everything that...,[3.0]
301,301,5,5_65,production of arms and weapons in this present...,[3.0]
302,302,5,5_151,Race to become more powerful can destroy the e...,[3.0]
303,303,5,5_404,In its attempt to harness the power of the ato...,[4.0]


In [10]:
df_test.to_csv(r'/content/test_prediction.csv', index = False)